In [1]:
import csv
from pathlib import Path

In [3]:
# read data

all_dict = []
with open('/home/ouyangkang/tmp.tsv', newline='') as f:
    reader = csv.DictReader(f, delimiter='\t')
    for row in reader:
        tmp_path = Path(row['Pathway'])
        fq_list = [i for i in tmp_path.glob('*.fq.gz')]
        if len(fq_list) > 2:
            fq_1 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '1.fq.gz']
            fq_2 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '2.fq.gz']
            fq_1.sort()
            fq_2.sort()
            write_dict_1 = {'Sample': row['Sample_name'], 'FQ1': fq_1[0], 'FQ2': fq_2[0]}
            write_dict_2 = {'Sample': row['Sample_name'], 'FQ1': fq_1[1], 'FQ2': fq_2[1]}
            all_dict += [write_dict_1]
            all_dict += [write_dict_2]

        else:
            fq_1 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '1.fq.gz']
            fq_2 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '2.fq.gz'] 
            write_dict = {'Sample': row['Sample_name'], 'FQ1': fq_1, 'FQ2': fq_2}
            all_dict += {write_dict}


# write a csv file
with open('/home/ouyangkang/result.csv', 'w', newline='') as s:

    fieldnames = ['Sample', 'FQ1', 'FQ2']
    writer = csv.DictWriter(s, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(all_dict)

/zfssz8/CNGB_DATA/BGISEQ01/DIPSEQ/DIPSEQT1/P24Z31300N0041_Temp/D2411320126/241216_63_DP8480012554TR_L01_D2411320126001
/zfssz8/CNGB_DATA/BGISEQ01/DIPSEQ/DIPSEQT1/P24Z31300N0041_Temp/D2411320127/241216_64_DP8480012555TR_L01_D2411320127001
/zfssz8/CNGB_DATA/BGISEQ01/DIPSEQ/DIPSEQT1/P24Z31300N0041_Temp/D2411320128/241216_64_DP8480012555TR_L01_D2411320128001


In [ ]:
for row in reader:
    print(row['Pathway'])
    fq_list = [i for i in Path(row['Pathway']).glob('*.fq.gz')]
    print(fq_list)

In [ ]:
# a python script for batch scRNA submitting in cloud platform

import csv
from pathlib import Path
import sys


input_file = sys.argv[1]
output_file = sys.argv[2]

all_dict = []
with open(input_file, 'r', newline='') as f:
    reader = csv.DictReader(f, fieldnames=['Seq_type', 'Sample_name', 'Pathway'], delimiter='\t')
    for row in reader:
        if 'cDNA' in row['Seq_type']:
            tmp_path = Path(row['Pathway'])
            fq_list = [i for i in tmp_path.glob('*.fq.gz')]
            if len(fq_list) > 2:
                cDNA_1 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '1.fq.gz']
                cDNA_2 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '2.fq.gz']
                cDNA_1.sort()
                cDNA_2.sort()
                write_dict_1 = {'Sample': row['Sample_name'], 'Type': 'cDNA', 'FQ1': cDNA_1[0], 'FQ2': cDNA_2[0]}
                write_dict_2 = {'Sample': row['Sample_name'], 'Type': 'cDNA', 'FQ1': cDNA_1[1], 'FQ2': cDNA_2[1]}
                all_dict.append(write_dict_1)
                all_dict.append(write_dict_2)
            else:
                fq_1 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '1.fq.gz'][0]
                fq_2 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '2.fq.gz'][0]
                write_dict = {'Sample': row['Sample_name'], 'Type': 'cDNA', 'FQ1': fq_1, 'FQ2': fq_2}
                all_dict.append(write_dict)
        else:
            tmp_path = Path(row['Pathway'])
            fq_list = [i for i in tmp_path.glob('*.fq.gz')]
            fq_1 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '1.fq.gz'][0]
            fq_2 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '2.fq.gz'][0]
            write_dict = {'Sample': row['Sample_name'], 'Type': 'Oligo', 'FQ1': fq_1, 'FQ2': fq_2}
            all_dict.append(write_dict)


result = {}
for item in all_dict:
    sample = item['Sample']
    if sample not in result:
        result[sample] = {
            'Sample': sample,
            'cDNA1': [],
            'cDNA2': [],
            'Oligo1': [],
            'Oligo2': [],
            'genomeDir': '',
            'Outdir': './',
            'expectcells': 10000,
            'forcecells': 'False',
            'Cpu': 8,
            'Mem': 120
        }
    if item['Type'] == 'Oligo':
        result[sample]['Oligo1'].extend(map(str, [item.get('FQ1', '')]))
        result[sample]['Oligo2'].extend(map(str, [item.get('FQ2', '')]))
    elif item['Type'] == 'cDNA':
        result[sample]['cDNA1'].extend(map(str, [item.get('FQ1', '')]))
        result[sample]['cDNA2'].extend(map(str, [item.get('FQ2', '')]))


# 写入输出文件
with open(output_file, 'w', newline='') as f:
    fieldnames = ['Sample', 'Oligo1', 'Oligo2', 'cDNA1', 'cDNA2', 'genomeDir', 'Outdir', 'expectcells', 'forcecells', 'Cpu', 'Mem']
    writer = csv.writer(f, delimiter=',')
    writer.writerow(fieldnames)

    for sample, data in result.items():
        max_row = max(len(data['cDNA1']), len(data['Oligo1']))
        for i in range(max_row):
            cDNA_1 = data['cDNA1'][i]
            cDNA_2 = data['cDNA2'][i]
            oligo_1 = data['Oligo1'][i] if i < len(data['Oligo1']) else None
            oligo_2 = data['Oligo2'][i] if i < len(data['Oligo2']) else None
            genomeDir = data['genomeDir']
            outDir = data['Outdir']
            expect_cell = data['expectcells']
            force_cell = data['forcecells']
            cpu = data['Cpu']
            mem = data['Mem']
            writer.writerow([sample, oligo_1, oligo_2, cDNA_1, cDNA_2, genomeDir, outDir, expect_cell, force_cell, cpu, mem])

In [3]:
from pathlib import Path
file_path = Path('/mnt/c')

print(str(file_path))

/mnt/c


In [ ]:
import csv
from pathlib import Path
import sys

input_file = sys.argv[1]
output_file = sys.argv[2]

# 读取输入文件并解析数据
all_dict = []
with open(input_file, 'r', newline='') as f:
    reader = csv.DictReader(f, fieldnames=['Seq_type', 'Sample_name', 'Pathway'], delimiter='\t')
    for row in reader:
        tmp_path = Path(row['Pathway'])
        fq_list = [i for i in tmp_path.glob('*.fq.gz')]
        fq_1 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '1.fq.gz']
        fq_2 = [i for i in fq_list if i.parts[-1].split('_')[-1] == '2.fq.gz']
        
        if 'cDNA' in row['Seq_type']:
            if len(fq_list) > 2:
                fq_1.sort()
                fq_2.sort()
                write_dict_1 = {'Sample': row['Sample_name'], 'Type': 'cDNA', 'FQ1': fq_1[0], 'FQ2': fq_2[0]}
                write_dict_2 = {'Sample': row['Sample_name'], 'Type': 'cDNA', 'FQ1': fq_1[1], 'FQ2': fq_2[1]}
                all_dict.append(write_dict_1)
                all_dict.append(write_dict_2)
            else:
                write_dict = {'Sample': row['Sample_name'], 'Type': 'cDNA', 'FQ1': fq_1[0], 'FQ2': fq_2[0]}
                all_dict.append(write_dict)
        else:
            write_dict = {'Sample': row['Sample_name'], 'Type': 'Oligo', 'FQ1': fq_1[0], 'FQ2': fq_2[0]}
            all_dict.append(write_dict)

# 按样本分组并合并数据
from collections import defaultdict

merged_data = defaultdict(lambda: {'Oligo1': None, 'Oligo2': None, 'cDNA1': None, 'cDNA2': None})
for entry in all_dict:
    sample = entry['Sample']
    if entry['Type'] == 'Oligo':
        merged_data[sample]['Oligo1'] = entry['FQ1']
        merged_data[sample]['Oligo2'] = entry['FQ2']
    elif entry['Type'] == 'cDNA':
        if not merged_data[sample]['cDNA1']:
            merged_data[sample]['cDNA1'] = entry['FQ1']
            merged_data[sample]['cDNA2'] = entry['FQ2']
        else:
            # 创建新行
            new_row = {'Sample': sample, 'Oligo1': None, 'Oligo2': None, 'cDNA1': entry['FQ1'], 'cDNA2': entry['FQ2']}
            merged_data[sample] = new_row

# 写入输出文件
with open(output_file, 'w', newline='') as f:
    fieldnames = ['Sample', 'Oligo1', 'Oligo2', 'cDNA1', 'cDNA2']
    writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()
    for sample, data in merged_data.items():
        writer.writerow(data)